In [1]:
import pandas as pd
import numpy as np
import gzip
import struct
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from adabound import AdaBound
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

file_dir = './Enzyme_Data/'
file_dir_gz = './Enzyme_Data/gzfile/'
#필요한 모듈

In [2]:
data_type = ">21000b"
label_type = ">b"

val_length = 44622
test_length = 44622
train_length = 137099
train_batch_size_value = 626 #데이터 크기 정하는곳
test_batch_size_value = 603 #데이터 크기 정하는곳

#train = 47, test = 67 이 나머지가 없는 값

In [3]:
train_seq = "deepec_prepare_train.idx3-ubyte"
train_label = "deepec_prepare_train.idx1-ubyte"

test_seq = "deepec_prepare_test.idx3-ubyte"
test_label = "deepec_prepare_test.idx1-ubyte"

val_seq = "deepec_prepare_val.idx3-ubyte"
val_label = "deepec_prepare_val.idx1-ubyte"

def data_loader(length, seq_file_name, label_file_name, seq_type, label_type):
    seq_data = open(file_dir + seq_file_name,'rb')
    seq_data.seek(16)
    
    label_data = open(file_dir + label_file_name,'rb')
    label_data.seek(8)
    
    while True:
        seq = struct.unpack(data_type, seq_data.read(21000))
        #1000 * 21 이기 때문에 21000byte씩 읽어 들임
        seq = np.array(seq)
        
        seq = torch.FloatTensor(seq)
        seq = torch.abs(seq)
        #절대값을 사용한 이유는 -1의 값이 나오기 때문에 abs로 절대값을 취해서 1의 값으로 변환해서 사용
        seq = seq.view(1,1000,21)
        
        label_value = struct.unpack(label_type, label_data.read(1))
        label_value = np.array(label_value, dtype = float)
        
        #BCELoss는 output과 값의 크기;가 동일하여야 하기 때문에 아래와 같이 사용 
        if label_value == 0:
            label_value = torch.FloatTensor([1.0, 0.0])
        else:
            label_value = torch.FloatTensor([0.0, 1.0])
            
        yield seq, label_value #yield함수를 사용하여 데이터 한개씩 return

#데이터 읽어 들이기

In [4]:
class TrainDataset(torch.utils.data.Dataset): 
    def __init__(self, length_t, seq_t, label_t):
        train_data = data_loader(length_t, seq_t, label_t, data_type, label_type)
        
    def __len__(self):
        return train_length
    
    def __getitem__(self, idx):
        data = next(train_data)
        x = data[0]
        y = data[1]    
        return x, y
    
class ValDataset(torch.utils.data.Dataset): 
    def __init__(self, length_t, seq_t, label_t):
        val_data = data_loader(length_t, seq_t, label_t, data_type, label_type)
        
    def __len__(self):
        return val_length
    
    def __getitem__(self, idx):
        data = next(val_data)
        x = data[0]
        y = data[1]    
        return x, y

class TestDataset(torch.utils.data.Dataset): 
    def __init__(self, length_t, seq_t, label_t):
        test_data = data_loader(length_t, seq_t, label_t, data_type, label_type)

    def __len__(self):
        return test_length

    def __getitem__(self, idx):
        data = next(test_data)
        x = data[0]
        y = data[1]    
        return x, y

    

train_data = data_loader(train_length, train_seq, train_label, data_type, label_type)
test_data = data_loader(test_length, test_seq, test_label, data_type, label_type)
val_data = data_loader(val_length, val_seq, val_label, data_type, label_type)

train_dataset = TrainDataset(train_length, train_seq, train_label)
val_dataset = ValDataset(val_length, val_seq, val_label)
test_dataset = TestDataset(test_length, test_seq, test_label)

traindataloader = DataLoader(train_dataset, batch_size = train_batch_size_value, shuffle = True, pin_memory = True, drop_last = True)
testdataloader = DataLoader(test_dataset, batch_size = test_batch_size_value, pin_memory = True, drop_last = True)
valdataloader = DataLoader(val_dataset, batch_size = test_batch_size_value, pin_memory = True, drop_last = True)

#pin_memory = 학습 도중 고정 메모리를 사용하여 학습 시간 및 정확도 향상을 위해 사용
#drop_last 는 배치사이즈만큼 사용시 데이터가 남는 현상이 있어 남는 데이터는 버리고 사용
#customdataloader 설정

In [5]:
class_weight = [5.48951105, 0.55010511]
#class_weight를 미리 계산한 값

In [6]:
#cuda() 붙여보기

class CNN1(nn.Module):
    
    def __init__(self):
        # 항상 torch.nn.Module을 상속받고 시작
        super(CNN1, self).__init__()

        eps_value = 1e-01
        momentum_value = 0.99
        bias_data = 0
        #다음은 uniform_
        conv1 = nn.Conv2d(1, 128, kernel_size = (4,21), stride = 1, dilation = 1) #케라스의 conv2d
        nn.init.xavier_uniform_(conv1.weight) #케라스의 conv2d와 설정을 똑같이 하기위해 사용 
        conv1.bias.data.fill_(bias_data) #케라스의 conv2d와 설정을 똑같이 하기위해 사용 
        
        batch_conv1 = nn.BatchNorm1d(128, momentum = momentum_value, eps = eps_value) #케라스의 batchnorm
        pool1 = nn.MaxPool2d(kernel_size=(997,1))  #케라스의 MaxPool2d

        self.CNN1_module = nn.Sequential(
            conv1,
            nn.ReLU(),
            pool1,
            nn.Flatten(),
            batch_conv1
        )
        
        conv2 = nn.Conv2d(1, 128, kernel_size = (8,21), stride = 1, dilation = 1)
        nn.init.xavier_uniform_(conv2.weight) 
        conv2.bias.data.fill_(bias_data)
        
        batch_conv2 = nn.BatchNorm1d(128, momentum = momentum_value, eps = eps_value)
        pool2 = nn.MaxPool2d(kernel_size=(993,1)) 

        self.CNN2_module = nn.Sequential(
            conv2,
            nn.ReLU(),
            pool2,
            nn.Flatten(),
            batch_conv2
        )
        
        conv3 = nn.Conv2d(1, 128, kernel_size = (16,21), stride = 1 , dilation = 1)
        nn.init.xavier_uniform_(conv3.weight)
        conv3.bias.data.fill_(bias_data)
        
        batch_conv3 = nn.BatchNorm1d(128, momentum = momentum_value, eps = eps_value)
        pool3 = nn.MaxPool2d(kernel_size=(985,1)) 
    
        self.CNN3_module = nn.Sequential(
            conv3,
            nn.ReLU(),
            pool3,
            nn.Flatten(),
            batch_conv3
        )

        
        fc1 = nn.Linear(384, 512) #케라스의 Dense
        batch_fc1 = nn.BatchNorm1d(512, momentum = momentum_value, eps = eps_value)
        nn.init.xavier_uniform_(fc1.weight) #케라스의 conv2d와 설정을 똑같이 하기위해 사용 
        fc1.bias.data.fill_(bias_data) #케라스의 conv2d와 설정을 똑같이 하기위해 사용 

        fc2 = nn.Linear(512, 512)
        batch_fc2 = nn.BatchNorm1d(512, momentum = momentum_value, eps = eps_value)
        nn.init.xavier_uniform_(fc2.weight)
        fc2.bias.data.fill_(bias_data)

        fc3 = nn.Linear(512, 2)
        batch_fc3 = nn.BatchNorm1d(2, momentum = momentum_value, eps = eps_value)
        nn.init.xavier_uniform_(fc3.weight)
        fc3.bias.data.fill_(bias_data)
        
 
        self.fc_module = nn.Sequential(
            fc1,
            batch_fc1,
            nn.ReLU(),
            fc2,
            batch_fc2,
            nn.ReLU(),
            fc3,
            batch_fc3
        )
        
        
    def forward(self, x):
        out1 = self.CNN1_module(x) # @16*4*4
        out2 = self.CNN2_module(x) # @16*4*4
        out3 = self.CNN3_module(x) # @16*4*4
        
        out = torch.cat((out1, out2, out3), dim = 1) # torch.cat은 케라스의 Concatenate
        
        out = self.fc_module(out)
        return out
    
#배치 정규화 균등분포

In [7]:
cnn = CNN1().cuda()
import time

In [ ]:
class_weight_value = torch.FloatTensor(class_weight).cuda()

criterion = nn.BCEWithLogitsLoss(weight = class_weight_value)

learning_rate = 0.0002

#optimizer = AdaBound(cnn.parameters(), lr = learning_rate, eps = 1e-08, final_lr=0.001)

optimizer = optim.Adam(cnn.parameters(), lr = learning_rate, eps = 1e-08)#, weight_decay = 1e-01)

epochs = 100

train_loss_list = []
test_loss_list = [] 
acc_list = [] 
non_acc_list = []
enz_acc_list = []
check_zero_pred_correct = 0
train_total_list = []
train_non_acc_list = []
train_enz_acc_list = []

for e in range(epochs):
    start_time = time.time()
    train_loss = 0
    val_loss = 0
    test_loss = 0
    
    test_zero_count = 0
    test_one_count = 0
    
    total_pred_correct = 0
    one_pred_correct = 0
    zero_pred_correct = 0
    
    train_zero_count = 0
    train_one_count = 0
    
    train_total_pred_correct = 0
    train_one_pred_correct = 0
    train_zero_pred_correct = 0

    train_data = data_loader(train_length, train_seq, train_label, data_type, label_type)
    test_data = data_loader(test_length, test_seq, test_label, data_type, label_type)
    val_data = data_loader(val_length, val_seq, val_label, data_type, label_type)
    
    for index, data in enumerate(traindataloader):
        cnn.train()
        input_data, label = data
       
        input_data = input_data.cuda(non_blocking = True)
        trian_label = label.cuda(non_blocking = True)
        
        optimizer.zero_grad()
        model_output = cnn(input_data)

        loss = criterion(model_output, trian_label)
        loss.backward()          
        optimizer.step()
        train_loss += loss.item()
        
        _, train_pred = torch.max(model_output.data, 1)
        
        for z in range(train_batch_size_value):
            if trian_label[z][0] ==  1:
                train_zero_count += 1
                if train_pred[z] == 0:
                    train_total_pred_correct += 1
                    train_zero_pred_correct += 1
                        
            elif trian_label[z][0] ==  0:
                train_one_count += 1
                if train_pred[z] == 1:
                    train_total_pred_correct += 1
                    train_one_pred_correct += 1

    with torch.no_grad(): # very very very very important!!!
        
        total_pred_correct = 0
        zero_pred_correct = 0
        one_pred_correct = 0
        
        for count, testdata in enumerate(testdataloader):
            cnn.eval()
            
            input_data, test_labels = testdata
            test_input_data = input_data.cuda(non_blocking = True)
            test_labels = test_labels.cuda(non_blocking = True)
            
            test_outputs = cnn(test_input_data)

            _, total_pred = torch.max(test_outputs.data, 1)
            
            loss = criterion(test_outputs, test_labels)
            test_loss += loss.item()
            
            for t in range(test_batch_size_value):
                if test_labels[t][0] ==  1:
                    test_zero_count += 1
                    if total_pred[t] == 0:
                        total_pred_correct += 1
                        zero_pred_correct += 1
                        
                elif test_labels[t][0] ==  0:
                    test_one_count += 1
                    if total_pred[t] == 1:
                        total_pred_correct += 1
                        one_pred_correct += 1
        
        check_zero_correct = float(100 * zero_pred_correct) / test_zero_count
        check_one_correct = float(100 * one_pred_correct) / test_one_count
        
        if check_zero_correct > 80:
            if check_one_correct > 90:
                print("model_save")
                torch.save(cnn.state_dict(), './saved_model/cnn1_adabound.pth')
        
        train_loss_list.append(train_loss/train_length)
        test_loss_list.append(test_loss/test_length)
        acc_list.append(float(100 * total_pred_correct) / test_length)
        non_acc_list.append(float(100 * zero_pred_correct) / test_zero_count)
        enz_acc_list.append(float(100 * one_pred_correct) / test_one_count)
        
        train_total_list.append(float(100 * train_total_pred_correct) / train_length)
        train_non_acc_list.append(float(100 * train_zero_pred_correct) / train_zero_count)
        train_enz_acc_list.append(float(100 * train_one_pred_correct) / train_one_count)
        
    print("epoch: {} | trn loss: {:.6f} | test loss: {:.6f}".format(e+1, train_loss/train_length, test_loss / test_length))
    print('Pred Total_acc : {:.2f} | Non_acc : {:.2f} | Enz : {:.2f}'.format(float(100 * total_pred_correct) / test_length, float(100 * zero_pred_correct) / test_zero_count, float(100 * one_pred_correct) / test_one_count))
    print('train Total_acc : {:.2f} | Non_acc : {:.2f} | Enz : {:.2f}'.format(float(100 * train_total_pred_correct) / train_length, float(100 * train_zero_pred_correct) / train_zero_count, float(100 * train_one_pred_correct) / train_one_count))
    print('작업 수행된 시간 : %f 초' % (time.time() - start_time))
  
    print("")
    
# 학습 및 평가

